![](../nci-logo.png)

-------
# Performance Examples of Using Data: Serial IO 


-----

## In this notebook:

<p>In this workshop we will explore the performance of accessing NetCDF4/HDF5 files by using NetCDF4-python and h5py modules. Several IO performance relating parameters, such as access pattern, storage layout, compression and shuffle etc. will be examined for serial IO.</p>

The NetCDF architecture is shown below:

<img src="images/netcdf_architecture.png" alt="Drawing" style="width: 600px;">

NetCDF4/HDF5 have many features like:
- Unique technology suite that makes possible the management of extremely large and complex data collections
- Unlimited size, extensibility, and portability
- General data model
- Unlimited variety of datatypes
- Flexible, efficient I/O
- Flexible data storage
- Data transformation and complex subsetting


#### The following material uses Geoscience Australia's Landsat 7 Data Collection which is available under the Create Commons License 4.0 through NCI's THREDDS Data Server. For more information on the collection and licensing, please [click here](http://geonetwork.nci.org.au/geonetwork/srv/eng/catalog.search#/metadata/f6600_8228_7170_1486). 

---------

<br>



## Prerequisites

#### Please load the following modules:

```
module load python/2.7.11
module load python/2.7.11-matplotlib
module load ipython/4.2.0-py2.7
module load netcdf4-python/1.2.4-py2.7

```

#### Then launch a new iPython (or Jupyter) notebook: 

`$ ipython notebook`


<br>

## Load necessary python modules

In [ ]:
from netCDF4 import Dataset
import numpy as np
import os
from tempfile import *
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import time
global tmp_file

## Useful Functions

#### Function view_prt : View the metadata of a variable named "varname" of a file named "filename"

In [ ]:
def view_prt(filename,varname='all'):
    f=Dataset(filename,'r')                      # Open the file named 'filename' for read
    print 'File Name: \t',filename               # Print the file name 
    print 'Format: \t',f.data_model              # Print the file format

    for item in f.dimensions:                    # Print all dimensions
        print 'dimension \t',f.dimensions[item].name, f.dimensions[item].size
    print '    '

    vars = f.variables.keys()
    for item in vars:
        if varname=='all' or item==varname:      # Print the selected metadata of all 
                                                 # variables or the variable named 'varname'
            print 'Variable: \t', item
            print 'Dimensions: \t', f[item].dimensions
            print 'Shape:    \t', f[item].shape
            print "size:  \t\t", f[item].size
            print "data type:  \t", f[item].dtype
            print "chunksize: \t", f[item].chunking()
            print "tendian: \t", f[item].endian()
            print "filters: \t", f[item].filters()
            print "ncattrs: \t", f[item].ncattrs() 
            print ""
    f.close()

#### Function read_var: Read and return data of a NetCDF4 variable named "varname" in a file named "filename"

In [ ]:
def read_var(filename,varname):
    fr = Dataset(filename,'r',format='NETCDF4')
    out=fr[varname][:]
    fr.close()
    return out

#### Function link_var: Link a variable named "varname" in a file named "filename". No actual data movement.

In [ ]:
def link_var(filename,varname):
    fr = Dataset(filename,'r',format='NETCDF4')
    return fr[varname]

#### Function create_file: Create a NetCDF4 files with 3 dimensions and a 3D variable named "varname" containing the data of 'val' array.
Dimensions are used to define the shape of data in netCDF.

In [ ]:
def create_file(d1,d1_name,d2,d2_name,d3,d3_name,val,varname,**args):
    global tmp_file 
    
    try:  # remove any existing temporary files.
        if os.path.exists(tmp_file.name):
            tmp_file.close()
            os.unlink(tmp_file.name)
    except:
        print " "

    tmp_file = NamedTemporaryFile(delete=False) # Create the temporary file object.
    fname=tmp_file.name                         # Get the temporary file name. 

    fw = Dataset(fname,'w',format='NETCDF4')    # Open the NetCDF file to write
    
    fw.createDimension(d1_name,len(d1))                 # Create the one-dimension named 'd1_name' 
                                                        # in the length of 'd1' array.
    dim_wrt=fw.createVariable(d1_name,d1.dtype,d1_name) # Create a 1D variable named 'd1_name' 
                                                        # with the data type of array 'd1' 
                                                        # and its dimension is 'd1_name'
    dim_wrt[:]=d1                                       # Write 'd1' data to dimension 'd1_name'

    
    
    fw.createDimension(d2_name,len(d2))                 # Create the one-dimension named 'd2_name'
                                                        # in the length of 'd2' array.
    dim_wrt=fw.createVariable(d2_name,d2.dtype,d2_name) # Create one-dimension variable named 'd2_name' 
                                                        # with the data type of array 'd2'
                                                        # and its dimension is 'd2_name'
    dim_wrt[:]=d2                                       # Write the 'd2' data to dimension 'd2_name'


    fw.createDimension(d3_name,len(d3))                 # Create the one-dimension named 'd3_name'
                                                        # in the length of 'd3' array.
    dim_wrt=fw.createVariable(d3_name,d3.dtype,d3_name) # Create one-dimension variable named 'd3_name' 
                                                        # with the data type of array 'd3'
                                                        # and its dimension is 'd3_name'
    dim_wrt[:]=d3                                       # Write the 'd3' data to dimension 'd3_name'
    

    var_wrt=fw.createVariable(
        varname,val.dtype,(d1_name,d2_name,d3_name),**args) # Create one-dimension variable named 'varname' 
                                                            # with the data type of array 'val' along
                                                            # 3 dimensions of 'd1_name, d2_name, d3_name'
    var_wrt[:]=val                                          # write the 'val' data to the variable 'varname'
    
    fw.close()                       # Close the file
    return tmp_file

#### Produce example data source of t, x, y and data array

In [ ]:
def get_data_src(nt=100,ny=100,nx=100):
    t = np.random.uniform(-1, 1, size=nt)
    y = np.random.uniform(-1, 1, size=ny)
    x= np.random.uniform(-1, 1, size=nx)
    data=np.random.uniform(-1, 1, size=nt*ny*nx)
    return t,y,x,data

## Remote vs. Local Access

For local files, this will be the filepath (i.e., /g/data...) while for remote access, this will be the OPeNDAP data URL.

In [ ]:
%%time
filename='http://dapds00.nci.org.au/thredds/dodsC/uc0/rs0_dev/multiple_band_variables/LS7_ETM_NBAR_P54_GANBAR01-002_089_078_2015_152_-26.nc'
varname='band1'
band=read_var(filename,varname)

In [ ]:
%%time
filename='/g/data2/uc0/rs0_dev/multiple_band_variables/LS7_ETM_NBAR_P54_GANBAR01-002_089_078_2015_152_-26.nc'
vaname='band1'
band=read_var(filename,varname)

## Create example 3D array in the size of (3600,60,60) along (t,y,x)

In [ ]:
t,y,x,src_data=get_data_src(nt=3600,ny=60,nx=60)

## Access Patterns
We can access the dataset in arbitary manner such as time series access, spatial access and block access. The IO performance relies on the number of IO operataions, which is arised from the coordination beteween the access pattern and the storage layout of the file. The 'locality' rule of access is that reads are generally faster when the data being accessed is all stored together.

<img src="images/fig1.png" alt="Drawing" style="width: 600px;"/>

We produce a variety of accesses from the time series one to the spatial one via different block accesses.

In [ ]:
blk_size=[{'name':'spatial_st','t':     1,'y':len(y),'x':len(x)},
          {'name':'block_t002','t':     2,'y':    30,'x':len(x)},
          {'name':'block_t003','t':     3,'y':    20,'x':len(x)},
          {'name':'block_t006','t':     6,'y':    10,'x':len(x)},
          {'name':'block_t012','t':    12,'y':    10,'x':    30},
          {'name':'block_t024','t':    24,'y':     5,'x':    30},  
          {'name':'block_t120','t':   120,'y':     1,'x':    30},
          {'name':'block_t180','t':   180,'y':     1,'x':    20},
          {'name':'block_t360','t':   360,'y':     1,'x':    10},
          {'name':'block_t600','t':   600,'y':     1,'x':     6},
          {'name':'block_t900','t':   900,'y':     1,'x':     4},
          {'name':'block_t1k2','t':  1200,'y':     1,'x':     3},
          {'name':'block_t1k8','t':  1800,'y':     1,'x':     2},
          {'name':'full_times','t':len(t),'y':     1,'x':     1}]

## Index-order storage layout

 The default storage layout for most file format is index-order layout, i.e. all the elements of multi-dimensional array are stored one after another along each dimension. Python and C use row-major ordering,that is, the 'fastest-varying' index is the last dimension.

<img src="images/tutr-lodset.png" alt="Drawing" style="width: 600px;"/>

For 3D array, the index-order data layout will store the data along x, y and t dimension respectively. Thus the 'spatial access' as above will be contiguous IO and the 'time series accesss' is non-contiguous IO with much more IO opeartions like seek, read, write etc. 

### TYX: Create index order storage layout TYX in a temporary file.

In [ ]:
# Create a file with the 3D variable named 'band'
f=create_file(t,'t',y,'y',x,'x',src_data,'band')
view_prt(f.name,'band')
# Print the metadata of 'band' variable in the above file.
band=link_var(f.name,'band')

### TYX: Full variable access if we have large enough memory

In [ ]:
t_start = time.clock()
band_subset = band[:,:,:]
t_end = time.clock() 
print "elapsed time is ",t_end-t_start,"Sec."

### TYX: Check the read time of different accesses

Split the data into several 'read' operations and each operation reads 'transfer size' of data to match the memory requirement.

In [ ]:

elap_ctg_tyx=[]
for item in blk_size:
# Count the total number of accessing.
    access_num=0
    t_size=item['t']
    y_size=item['y']
    x_size=item['x']
# Read the whole variable data in multiple read operations.
    t_start = time.clock() 
    for it in range(len(t)/t_size):
        itbeg=it*t_size
        for iy in range(len(y)/y_size):
            iybeg=iy*y_size
            for ix in range(len(x)/x_size):
                ixbeg=ix*x_size
                band_subset = band[itbeg:itbeg+t_size,iybeg:iybeg+y_size,ixbeg:ixbeg+x_size]
                access_num+=1
    t_end = time.clock()
    print item['name'], " elapsed time is ",(t_end-t_start),"Sec.", " access time is",access_num
    elap_ctg_tyx.append(t_end-t_start)
    

<p> For non-contiguous IO, number of disk accesses that make I/O slow, not the number of values read. </p> 

### YXT: Create index order storage layout YXT in a temporary file.

<p> If we instead want the time series to be quick, we can reorganize the data so x or y is the most slowly varying dimension and time varies fastest, resulting in fast time-series access and slow spatial access. In either case, the slow access is so slow that it makes the data essentially inaccessible for all practical purposes, e.g. in analysis or visualization. <p>

NCO (netCDF operators) program ncpdq ("permute dimensions quickly") can be used to exchange dimensions.

In [ ]:
# Create the variable along (y,x,t)
f=create_file(y,'y',x,'x',t,'t',src_data,'band')
view_prt(f.name,'band')
band=link_var(f.name,'band')

### YXT: Full Access if we have large enough memory

In [ ]:
#%%time
t_start = time.clock() 
band_subset = band[:,:,:]
t_end = time.clock() 
print "elapsed time is ",(t_end-t_start),"Sec."

### YXT: Check the read time of different accesses

In [ ]:

elap_ctg_yxt=[]
for item in blk_size:
# Count the total number of accessing.
    access_num=0
    t_size=item['t']
    y_size=item['y']
    x_size=item['x']
# Read the whole variable data in multiple read operations.
    t_start = time.clock() 
    for iy in range(len(y)/y_size):
        iybeg=iy*y_size
        for ix in range(len(x)/x_size):
            ixbeg=ix*x_size
            for it in range(len(t)/t_size):
                itbeg=it*t_size
                band_subset = band[iybeg:iybeg+y_size,ixbeg:ixbeg+x_size,itbeg:itbeg+t_size]
                access_num+=1
    t_end = time.clock()
    print item['name'], " elapsed time is ",(t_end-t_start),"Sec.", " access time is",access_num    
    elap_ctg_yxt.append(t_end-t_start)

## Chunked storage layout

<p> A better solution is the use of chunking, storing multidimensional data in multi-dimensional rectangular chunks to speed up slow accesses at the cost of slowing down fast accesses. Programs that access chunked data can be oblivious to whether or how chunking is used. <p>

<img src="images/chunking.png" alt="Drawing" style="width: 600px;"/>|

Let's consider these selections

<img src="images/contiguous_seek.png" alt="Drawing" style="width: 400px;"/>
<p>If index-order: 2 seeks</p>
If chunked: 10 seeks

<img src="images/chunk_seek.png" alt="Drawing" style="width: 400px;"/>
<p>if index-order: 16 seeks</p>
if chunked: 4 seeks

<p> For 2-dimensional data, we could support equally frequent access by either rows or columns by chunking the data into rectangular chunks (or tiles) so that reading a row requires the same number of disk accesses as reading a column. 
Note each chunk is a disk block that must be read completely to access any of its data values. An optimum solution is to make the chunks similar in shape to the entire array, so that the same number of chunks are required to read an entire row or an entire column. </p>

The appropriate size of chunking depends on access patterns and hardware itself, like disk cache sizes, etc. When writing or reading, try to use hyperslab selections that coincide with chunk boundaries.

The process of picking a chunk shape is a trade-off between the following constraints:

<ul>
<li> Larger chunks for a given dataset size reduce the size of the chunk B-tree, making it faster to find and load chunks. </li>
<li> Since chunks are all or nothing (reading a portion loads the entire chunk), larger chunks also increase the chance that youâll read data into memory you wonât use. </li>
<li> Chunk cache can only hold a finite number of chunks. Large chunk can not fit into the chunk cache. Entire chunk has to be in memory and may cause OS to page memory to disk, slowing down the entire system. </li>
<li> Small chunks may create large amount of metadata to fill the disk space and extra time will be spent to look up each chunk. More IO operations will be invloved as each chunk is stored independently. A good rule of thumb for most datasets is to keep chunks above 10KiB or so. </li>
</ul>

<p>When writing or reading, try to use hyperslab selections that coincide with chunk boundaries.</p>

### TYX: Chunk number=20

In [ ]:
# Specify the transfer size along each dimension
chunk_number=20

# Create the chunk shape. Make the number of chunks for z
chunk_size = (len(t)/(chunk_number*chunk_number),len(y)/chunk_number,len(x)/chunk_number)

# Create a chunked NetCDF4 file
f=create_file(t,'t',y,'y',x,'x',src_data,'band',chunksizes=chunk_size)

view_prt(f.name,'band')
band=link_var(f.name,'band')

In [ ]:

elap_ck20_tyx=[]
for item in blk_size:
# Count the total number of accessing.
    access_num=0
    t_size=item['t']
    y_size=item['y']
    x_size=item['x']
# Read the whole variable data in multiple read operations.
    t_start = time.clock() 
    for it in range(len(t)/t_size):
        itbeg=it*t_size
        for iy in range(len(y)/y_size):
            iybeg=iy*y_size
            for ix in range(len(x)/x_size):
                ixbeg=ix*x_size
                band_subset = band[itbeg:itbeg+t_size,iybeg:iybeg+y_size,ixbeg:ixbeg+x_size]
                access_num+=1
    t_end = time.clock()
    print item['name'], " elapsed time is ",(t_end-t_start),"Sec.", " access time is",access_num
    elap_ck20_tyx.append(t_end-t_start)

### TYX: Chunk number=10

In [ ]:
# Specify the transfer size along each dimension
chunk_number=10
# Create the chunk shape. Make the number of chunks for z
chunk_size = (len(t)/(chunk_number*chunk_number),len(y)/chunk_number,len(x)/chunk_number)
# Create a chunked NetCDF4 file
f=create_file(t,'t',y,'y',x,'x',src_data,'band',chunksizes=chunk_size)

view_prt(f.name,'band')
band=link_var(f.name,'band')

elap_ck10_tyx=[]
for item in blk_size:
# Count the total number of accessing.
    access_num=0
    t_size=item['t']
    y_size=item['y']
    x_size=item['x']
# Read the whole variable data in multiple read operations.
    t_start = time.clock() 
    for it in range(len(t)/t_size):
        itbeg=it*t_size
        for iy in range(len(y)/y_size):
            iybeg=iy*y_size
            for ix in range(len(x)/x_size):
                ixbeg=ix*x_size
                band_subset = band[itbeg:itbeg+t_size,iybeg:iybeg+y_size,ixbeg:ixbeg+x_size]
                access_num+=1
    t_end = time.clock()
    print item['name'], " elapsed time is ",(t_end-t_start),"Sec.", " access time is",access_num
    elap_ck10_tyx.append(t_end-t_start)

### TYX: Block access on matched chunk layout

In [ ]:
tblk=2
yblk=30
xblk=60
chunk_size = [tblk,yblk,xblk]
f=create_file(t,'t',y,'y',x,'x',src_data,'band',chunksizes=chunk_size)
band=link_var(f.name,'band')
t_start = time.clock() 
for it in range(len(t)/tblk):
    itbeg=it*tblk
    for iy in range(len(y)/yblk):
        iybeg=iy*yblk
        for ix in range(len(x)/xblk):
            ixbeg=ix*xblk
#            print itbeg,iybeg,ixbeg
            band_subset = band[itbeg:itbeg+tblk-1,iybeg:iybeg+yblk-1,ixbeg:ixbeg+xblk-1]
t_end = time.clock() 
print "elapsed time is ",(t_end-t_start),"Sec."

In [ ]:
%matplotlib inline
data=[elap_ctg_tyx,elap_ctg_yxt,elap_ck20_tyx,elap_ck10_tyx]
labels=['contiguous_tyx','contiguous_yxt','chunk20_tyx','chunk10_tyx']

fig,axes=plt.subplots(nrows=1,ncols=1)
bplot=axes.boxplot(data,vert=True,patch_artist=True)
colors = ['pink', 'lightblue', 'lightgreen','lightyellow']
for patch, color in zip(bplot['boxes'], colors):
    patch.set_facecolor(color)
plt.grid(True)   
plt.ylabel('Read Time (s)')
axes.set_yscale("log")
plt.setp(axes, xticks=[y+1 for y in range(len(data))],
         xticklabels=labels)
plt.show()

# Compression

 If the data is compressed in netCDF-4 or HDF5, it has to be chunked, because a chunk is the atomic unit of compression as well as disk access. There is no need to decompress the whole file when reading a block data.

#### Chunk scale = 20

In [ ]:
t,y,x,band2=get_data_src(nt=400,ny=100,nx=100)
chunk_scale=20
chunk_size = (len(t)/(chunk_scale*chunk_scale),len(y)/chunk_scale,len(x)/chunk_scale)
print 'chunk size is ',chunk_size
cmp_time_cs20=[]
cmp_size_cs20=[]
cmp_label_cs20=[]
print '{:>20}{:>20}{:>20}'.format('Comp. Level','Write Time(s)', 'File Size(MB)')
for ilevel in range(0,10):
    start = time.clock() 
    f=create_file(t,'t',y,'y',x,'x',band2,'band2',chunksizes=chunk_size,complevel=ilevel,zlib=True)
    elapsed = time.clock()
    file_time = elapsed - start
    file_size = os.path.getsize(f.name)/1048576.0
    cmp_time_cs20.append(file_time)
    cmp_size_cs20.append(file_size)
    cmp_label_cs20.append(str(ilevel))
    print '{:>20d}{:>20.2f}{:>20.2f}'.format(ilevel,file_time,file_size)

#### Chunk scale = 10

In [ ]:
chunk_scale=10
chunk_size = (len(t)/(chunk_scale*chunk_scale),len(y)/chunk_scale,len(x)/chunk_scale)
print 'chunk size is ',chunk_size

cmp_time_cs10=[]
cmp_size_cs10=[]
cmp_label_cs10=[]

print '{:>20}{:>20}{:>20}'.format('Comp. Level','Write Time(s)', 'File Size(MB)')
for ilevel in range(0,10):
    start = time.clock() 
    f=create_file(t,'t',y,'y',x,'x',band2,'band2',chunksizes=chunk_size,complevel=ilevel,zlib=True)
    elapsed = time.clock()
    file_time = elapsed - start
    file_size = os.path.getsize(f.name)/1048576.0
    cmp_time_cs10.append(file_time)
    cmp_size_cs10.append(file_size)
    cmp_label_cs10.append(str(ilevel))
    print '{:>20d}{:>20.2f}{:>20.2f}'.format(ilevel,file_time,file_size)

#### Chunk scale = 5

In [ ]:
chunk_scale=5
chunk_size = (len(t)/(chunk_scale*chunk_scale),len(y)/chunk_scale,len(x)/chunk_scale)
print 'chunk size is ',chunk_size
cmp_time_cs05=[]
cmp_size_cs05=[]
cmp_label_cs05=[]
print '{:>20}{:>20}{:>20}'.format('Comp. Level','Write Time(s)', 'File Size(MB)')
for ilevel in range(0,10):
    start = time.clock() 
    f=create_file(t,'t',y,'y',x,'x',band2,'band2',chunksizes=chunk_size,complevel=ilevel,zlib=True)
    elapsed = time.clock()
    file_time = elapsed - start
    file_size = os.path.getsize(f.name)/1048576.0
    cmp_time_cs05.append(file_time)
    cmp_size_cs05.append(file_size)
    cmp_label_cs05.append(str(ilevel))
    print '{:>20d}{:>20.2f}{:>20.2f}'.format(ilevel,file_time,file_size)

In [ ]:
%matplotlib inline  
fig, ax = plt.subplots()
ax.plot(cmp_time_cs20, cmp_size_cs20,'.-',label='chunk scale=20')
ax.plot(cmp_time_cs10, cmp_size_cs10,'.-',label="chunk scale=10")
ax.plot(cmp_time_cs05, cmp_size_cs05,'.-',label="chunk scale= 5")

plt.title('Compression Benchmark for 3D Random Data')
ax.set_ylabel('File Size (MiB)')
ax.set_xlabel('Compression Time (s)')
ax.legend(loc='best')
plt.show()

The Landsat compression Benchmak is shown below

<img src="images/compress_time.png" alt="Drawing" style="width: 400px;"/>

In [ ]:
view_prt(f.name,'band2')

#### Effect of shuffle

The SHUFFLE filter repacks the data in the chunk so that all the first bytes of the integers
are together, then all the second bytes, and so on. For dictionary-based compressors like GZIP and LZF, it is much more efficient to compress
long runs of identical values, like all the zero values collected from the first two
bytes of the dataset integers. There are also savings from the repeated elements at the
third byte position. Only the fourth byte position looks really hard to compress.

In [ ]:
chunk_scale=10
chunk_size = (len(t)/(chunk_scale*chunk_scale),len(y)/chunk_scale,len(x)/chunk_scale)
print 'chunk size is ',chunk_size

cmp_time_nosuf=[]
cmp_size_nosuf=[]
cmp_label_nosuf=[]

print '{:>20}{:>20}{:>20}'.format('Comp. Level','Write Time(s)', 'File Size(MB)')
for ilevel in range(0,10):
    start = time.clock() 
    f=create_file(t,'t',y,'y',x,'x',band2,'band2',chunksizes=chunk_size,complevel=ilevel,zlib=True,shuffle=False)
    elapsed = time.clock()
    file_time = elapsed - start
    file_size = os.path.getsize(f.name)/1048576.0
    cmp_time_nosuf.append(file_time)
    cmp_size_nosuf.append(file_size)
    cmp_label_nosuf.append(str(ilevel))
    print '{:>20d}{:>20.2f}{:>20.2f}'.format(ilevel,file_time,file_size)

In [ ]:
view_prt(f.name,'band2')

In [ ]:
%matplotlib inline  
fig, ax = plt.subplots()
ax.plot(cmp_time_cs10, cmp_size_cs10,'.-',label='shuffle')
ax.plot(cmp_time_nosuf, cmp_size_nosuf,'.-',label='no-shuffle')
plt.title('Compression Benchmark for 3D Random Data')
plt.legend(loc='best')
ax.set_ylabel('File Size (MiB)')
ax.set_xlabel('Compression Time (s)')
plt.show()

# References

<ul>
<li> 'Parallel HDF5',Quincey Koziol, The HDF group, 2015.</li>
<li> 'Portable Parallel I/O: Handling large datasets in heterogeneous parallel environments', Michael Stephan, JULICH, 2013.</li>
<li> 'Python and HDF5',Andrew Collette, 2014.</li>
<li> 'Chunking Data: Why it Matters', http://www.unidata.ucar.edu/blogs/developer/entry/chunking_data_why_it_matters, 2013 </li>
</ul>